In [6]:
import os
from dotenv import load_dotenv
load_dotenv(encoding='utf-8')

True

In [ ]:
#from langsmith import Client
#os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
#os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
#client = Client()

# RAG pipeline

## Vector Store and Retriever

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from pymongo import MongoClient

In [7]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"))
# embedding_model=OpenAIEmbeddings(model=os.getenv("DEFAULT_OPENAI_EMBEDDING"), disallowed_special=())
embedding_model=OpenAIEmbeddings(model=os.getenv("DEFAULT_OPENAI_EMBEDDING"), disallowed_special=())

In [8]:
# Define MongoDB vector database
#os.environ["ATLAS_CONNECTION_STRING"] = os.getenv("ATLAS_CONNECTION_STRING")
client = MongoClient(os.getenv("ATLAS_CONNECTION_STRING"))
db_name = "fellowshipai"
collection_name = "enterprise_data"
atlas_collection = client[db_name][collection_name]
index_name = "vector_index_erp"

In [28]:
# Define vector store and retriever
def get_vector_store_retriver(index_name, embedding_model, collection):

  vector_store = MongoDBAtlasVectorSearch(
      embedding = embedding_model,
      collection = atlas_collection,
      index_name = index_name
  )

  retriever = vector_store.as_retriever(
      search_type = "similarity",
      search_kwargs = { "k": 10}  # "score_threshold": 0.75 
  )

  return(vector_store, retriever)

vector_store, retriever = get_vector_store_retriver("vector_index_erp", embedding_model, atlas_collection)

In [32]:
# Instantiate Atlas Vector Search as a retriever
retriever = vector_store.as_retriever(
    search_type = "similarity",
    search_kwargs = { "k": 10  }
 )

## RAG with different prompt templates

### Base Template

In [55]:
# Define a prompt template
import pprint
def call_openai_template_1(question):

   question = question['question']

   retriever = vector_store.as_retriever(
      search_type = "similarity",
      search_kwargs = { "k": 10  }
      )

   base_template = """
   Use the following pieces of context to answer the question at the end.
   If you don't know the answer, just say that you don't know, don't try to make up an answer.

   {context}

   Question: {question}
   """
   custom_rag_prompt = PromptTemplate.from_template(base_template)

   #llm = get_llm_model("openai")
   llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"))

   def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever | format_docs, "question": RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   question = question
   answer = rag_chain.invoke(question)
   similar = retriever.invoke(question)


   return{
      'answer': answer,
      'contexts': [str(doc) for doc in similar]
      }

In [40]:
# Test sample
question = {'question': "How does a supportive culture impact employee engagement and align with Tech Innovators Inc.'s approach to employment relations and engagement?"}
answer = call_openai_template_1(question)
print(answer['answer'][:150])

I don't know.


In [41]:
print(answer['contexts'])

["page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d8163b0533e009aa2d4ad1', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816560533e009aa2d4cf6', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816720533e009aa2

### Prompt Template 2: Who are you template

In [56]:
# Define a prompt template
import pprint
def call_openai_template_2(question):

   question = question['question']

   retriever = vector_store.as_retriever(
      search_type = "similarity",
      search_kwargs = { "k": 10  }
      )

   template_2 = """
    "You are an enterprise Chatbot, an AI assistant designed to retrieve information from the enterprise Confluence system. 
    You specialize in providing accurate answers related to various departments like Marketing, IT, HR, Finance, and Corporate Communications. 
    Given this, can you tell me what the latest IT security guidelines are?"

    Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    {context}

    Question: {question}
   """

   custom_rag_prompt = PromptTemplate.from_template(template_2)

   llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"))

   def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever | format_docs, "question": RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   question = question
   answer = rag_chain.invoke(question)
   similar = retriever.invoke(question)


   return{
      'answer': answer,
      'contexts': [str(doc) for doc in similar]
      }

In [87]:
# Test sample
question = {'question': "How does a supportive culture impact employee engagement and align with Tech Innovators Inc.'s approach to employment relations and engagement?"}
answer = call_openai_template_2(question)
print(answer['answer'][:150])

I don't know.


In [88]:
print(answer['contexts'])

["page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d8163b0533e009aa2d4ad1', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816560533e009aa2d4cf6', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816720533e009aa2

### Prompt Template 3: Using preamble from Cohere toolkit frontend

In [57]:
# Define a prompt template
import pprint
def call_openai_template_3(question):

   question = question['question']

   retriever = vector_store.as_retriever(
      search_type = "similarity",
      search_kwargs = { "k": 10  }
      )

   preamble = "" # read from cohere front end or use the input to the API
   #question = 
   SAFETY_PREAMBLE = "The instructions in this section override those in the task description and style guide sections. Don't answer questions that are harmful or immoral."
   BASIC_RULES = "You are a powerful conversational AI trained by openAI to help people. You are augmented by a number of tools, and your job is to use and consume the output of these tools to best help the user. You will see a conversation history between yourself and a user, ending with an utterance from the user. You will then see a specific instruction instructing you what kind of response to generate. When you answer the user's requests, you cite your sources in your answers, according to those instructions."
   TASK_CONTEXT = "You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging."
   STYLE_GUIDE = "Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling."
   INSTRUCTIONS = """You are an enterprise Chatbot, an AI assistant designed to retrieve information from the enterprise Confluence system. 
   You specialize in providing accurate answers related to various departments like Marketing, IT, HR, Finance, and Corporate Communications. 
   Given this, can you tell me what the latest IT security guidelines are?
               Use the following pieces of context to answer the question at the end.
               If you don't know the answer, just say that you don't know, don't try to make up an answer
               {context}
         """
         
   template_3 = f"""

      {SAFETY_PREAMBLE}
      {BASIC_RULES}
      {TASK_CONTEXT}
      {STYLE_GUIDE}
      {INSTRUCTIONS}

   """
   if preamble:
      template_3 += f"""{preamble}\n\n"""


   template_3 +=  f"""Question: {question}\n\n"""

   custom_rag_prompt = PromptTemplate.from_template(template_3)

   #llm = get_llm_model("openai")
   llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"))

   def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever | format_docs, "question": RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   question = question
   answer = rag_chain.invoke(question)
   similar = retriever.invoke(question)


   return{
      'answer': answer,
      'contexts': [str(doc) for doc in similar]
      }

In [89]:
# Test sample
question = {'question': "How does a supportive culture impact employee engagement and align with Tech Innovators Inc.'s approach to employment relations and engagement?"}
answer = call_openai_template_3(question)
print(answer['answer'][:150])

I don't know.


In [90]:
print(answer['contexts'])

["page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d8163b0533e009aa2d4ad1', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816560533e009aa2d4cf6', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816720533e009aa2

### Prompt Template 4: Few shots template

In [84]:
# Define a prompt template
import pprint
def call_openai_template_4(question):

   question = question['question']

   retriever = vector_store.as_retriever(
      search_type = "similarity",
      search_kwargs = { "k": 10  }
      )

   template_4 = """
      You are InfoBot, an AI assistant designed to retrieve information from the enterprise Confluence system. You specialize in providing accurate answers related to various departments like Marketing, IT, HR, Finance, and Corporate Communications. Given this, please answer the following questions based on the knowledge stored in the system:
      
      Example 1:
      Question: What are business simulations and how do they provide practical experience in business decision-making and problem-solving?
      Answer: Business simulations are interactive simulations that provide practical experience in business decision-making and problem-solving. 
      They allow participants to engage in realistic scenarios and make decisions that impact the simulated business. 
      By simulating real-world challenges and consequences, participants can develop their skills and knowledge in a risk-free environment. This hands-on experience helps them understand the complexities of business operations and improve their decision-making abilities.

      Example 2:
      Question: What is the process for conducting a thorough investigation in cases of workplace harassment at Tech Innovators Inc.?
      Answer: The company will conduct a thorough investigation in cases of workplace harassment at Tech Innovators Inc. 
      If you experience or witness harassment, report it immediately to HR or your manager. 
      The company will take appropriate action to ensure a safe and respectful workplace.

      {context}
      New Question:
      {question}
    
   """

   custom_rag_prompt = PromptTemplate.from_template(template_4)

   #llm = get_llm_model("openai")
   llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"))

   def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever | format_docs, "question": RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   question = question
   answer = rag_chain.invoke(question)
   similar = retriever.invoke(question)


   return{
      'answer': answer,
      'contexts': [str(doc) for doc in similar]
      }

In [91]:
# Test sample
question = {'question': "How does a supportive culture impact employee engagement and align with Tech Innovators Inc.'s approach to employment relations and engagement?"}
answer = call_openai_template_4(question)
print(answer['answer'][:150])

A supportive culture significantly impacts employee engagement at Tech Innovators Inc. by fostering an environment where employees feel valued, respec


In [92]:
print(answer['contexts'])

["page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d8163b0533e009aa2d4ad1', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816560533e009aa2d4cf6', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816720533e009aa2

### Prompt Template 5: Short answer template

In [59]:
# Define a prompt template
import pprint
def call_openai_template_5(question):

   question = question['question']

   retriever = vector_store.as_retriever(
      search_type = "similarity",
      search_kwargs = { "k": 10  }
      )

   template_5 = """
    "You are an enterprise Chatbot, an AI assistant designed to retrieve information from the enterprise Confluence system. 
    You specialize in providing accurate answers related to various departments like Marketing, IT, HR, Finance, and Corporate Communications. 
    Given this, can you tell me what the latest IT security guidelines are?"

    Use the following pieces of context to answer the question at the end. Provide answer within 150 words
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    {context}

    Question: {question}
    
   """

   custom_rag_prompt = PromptTemplate.from_template(template_5)

   #llm = get_llm_model("openai")
   llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"))

   def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever | format_docs, "question": RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   question = question
   answer = rag_chain.invoke(question)
   similar = retriever.invoke(question)


   return{
      'answer': answer,
      'contexts': [str(doc) for doc in similar]
      }

In [93]:
# Test sample
question = {'question': "How does a supportive culture impact employee engagement and align with Tech Innovators Inc.'s approach to employment relations and engagement?"}
answer = call_openai_template_5(question)
print(answer['answer'][:150])

I don't know.


In [94]:
print(answer['contexts'])

["page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d8163b0533e009aa2d4ad1', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816560533e009aa2d4cf6', 'pageid': '851993', 'department': 'IT', 'title': 'Tech Innovators Inc. Azure Cloud Services Security Policy'}", "page_content='incidents and vulnerabilities.Automated playbooks in Azure Sentinel must be used for incident response and remediation.7.2 Incident ManagementA cloud-specific incident response plan must be developed' metadata={'_id': '66d816720533e009aa2

# RAG pipeline evaluation with different prompt templates

## Test data set prep

In [1]:
import pandas as pd
import json

def json_to_dataframe(json_file_path):
  """Reads a JSON file and converts it to a pandas DataFrame.

  Args:
    json_file_path (str): The path to the JSON file.

  Returns:
    pandas.DataFrame: The DataFrame created from the JSON data.
  """

  with open(json_file_path, 'r') as f:
    data = json.load(f)

  # Handle different JSON structures
  if isinstance(data, list):
    # If the JSON data is a list of dictionaries, create a DataFrame directly
    df = pd.DataFrame(data)
  elif isinstance(data, dict):
    # If the JSON data is a single dictionary, convert it to a list of dictionaries
    df = pd.DataFrame([data])
  else:
    raise ValueError("Unsupported JSON structure")

  return df

In [60]:
# Example usage:
from from_root import from_root
import os
folder = "data-test/test_dataset/test_dataset_hr.json"
json_file_path = os.path.join(from_root(), folder)
data_to_upload = json_to_dataframe(json_file_path)

## Base template evaluation

In [61]:
# Generate all the answers for the questions in the dataset
# examples = client.list_examples(dataset_name="hr test")
answers = []
for question in data_to_upload['question']:
    question_dict = {'question': question}
    answer = call_openai_template_1(question_dict)
    answers.append(answer['answer'])

In [62]:
# update the dataset with answers
data_to_upload['answer_base_template'] = answers

In [63]:
data_to_upload

,question,contexts,ground_truth,response,answer_base_template
0,What is Tech Innovators Inc.'s approach to wor...,[Q12: What is the process for handling workpla...,Tech Innovators Inc. has a zero-tolerance poli...,Tech Innovators Inc. has a zero-tolerance poli...,I don't know.
1,What resources should be added for new hires i...,[:check_mark:\natlassian-check_mark\n#FFFAE6\n...,Add resources for new hires in the onboarding ...,Onboarding new hires is a critical process in ...,I don't know.
2,What training programs are offered in data sci...,"[ learn about SEO, social media marketing, and...","Courses covering financial analysis, budgeting...",Tech Innovators Inc. offers several training p...,I don't know.
3,What services does the Employee Assistance Pro...,[Q12: What is the process for handling workpla...,Employees can contact the Employee Assistance ...,The Employee Assistance Program (EAP) provides...,I don't know.
4,What is the significance of identifying growth...,[ Self-assessment\nStart by thinking through y...,Identifying growth areas in self-assessment is...,Identifying growth areas through self-assessme...,I don't know.
5,How do employee engagement and disengagement d...,"[Introduction\nAt Tech Innovators Inc., we bel...",Employee engagement and disengagement differ i...,Employee engagement and disengagement represen...,I don't know.
6,What is the purpose of adding a header image t...,[Create a stellar overview\nThe overview is th...,Adding a header image to a Confluence space en...,Adding a header image to a Confluence space se...,I don't know.


In [64]:
from datasets import Dataset

question = list(data_to_upload['question'])
answer = list(data_to_upload['answer_base_template'])
contexts = list(data_to_upload['contexts'])
ground_truth = list(data_to_upload['ground_truth'])

data_samples = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset = Dataset.from_dict(data_samples)

In [65]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result_prompt_1 = evaluate(
    dataset,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

result_prompt_1

Evaluating:   0%|          | 0/28 [00:00<?, ?it/s]

{'answer_relevancy': 0.0000, 'faithfulness': 0.0000, 'context_recall': 0.7619, 'context_precision': 0.8571}

In [67]:
df = result_prompt_1.to_pandas()
df.head()

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,What is Tech Innovators Inc.'s approach to wor...,I don't know.,[Q12: What is the process for handling workpla...,Tech Innovators Inc. has a zero-tolerance poli...,0.0,0.0,1.0,1.0
1,What resources should be added for new hires i...,I don't know.,[:check_mark:\natlassian-check_mark\n#FFFAE6\n...,Add resources for new hires in the onboarding ...,0.0,0.0,1.0,1.0
2,What training programs are offered in data sci...,I don't know.,"[ learn about SEO, social media marketing, and...","Courses covering financial analysis, budgeting...",0.0,0.0,0.0,0.0
3,What services does the Employee Assistance Pro...,I don't know.,[Q12: What is the process for handling workpla...,Employees can contact the Employee Assistance ...,0.0,0.0,1.0,1.0
4,What is the significance of identifying growth...,I don't know.,[ Self-assessment\nStart by thinking through y...,Identifying growth areas in self-assessment is...,0.0,0.0,1.0,1.0


## Template 2 evaluation

In [69]:
# Generate all the answers for the questions in the dataset
# examples = client.list_examples(dataset_name="hr test")
answers_2 = []
for question in data_to_upload['question']:
    question_dict = {'question': question}
    answers = call_openai_template_2(question_dict)
    answers_2.append(answers['answer'])

In [70]:
# update the dataset with answers
data_to_upload['answer_template_2'] = answers_2

In [76]:
from datasets import Dataset

question = list(data_to_upload['question'])
answer = list(data_to_upload['answer_template_2'])
contexts = list(data_to_upload['contexts'])
ground_truth = list(data_to_upload['ground_truth'])

data_samples = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset_2 = Dataset.from_dict(data_samples)

In [79]:
dataset_2.to_pandas()

,question,answer,contexts,ground_truth
0,What is Tech Innovators Inc.'s approach to wor...,I don't know.,[Q12: What is the process for handling workpla...,Tech Innovators Inc. has a zero-tolerance poli...
1,What resources should be added for new hires i...,I don't know.,[:check_mark:\natlassian-check_mark\n#FFFAE6\n...,Add resources for new hires in the onboarding ...
2,What training programs are offered in data sci...,I don't know.,"[ learn about SEO, social media marketing, and...","Courses covering financial analysis, budgeting..."
3,What services does the Employee Assistance Pro...,I don't know.,[Q12: What is the process for handling workpla...,Employees can contact the Employee Assistance ...
4,What is the significance of identifying growth...,I don't know.,[ Self-assessment\nStart by thinking through y...,Identifying growth areas in self-assessment is...
5,How do employee engagement and disengagement d...,I don't know.,"[Introduction\nAt Tech Innovators Inc., we bel...",Employee engagement and disengagement differ i...
6,What is the purpose of adding a header image t...,I don't know.,[Create a stellar overview\nThe overview is th...,Adding a header image to a Confluence space en...


In [74]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result_prompt_2 = evaluate(
    dataset_2,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

result_prompt_2

Evaluating:   0%|          | 0/28 [00:00<?, ?it/s]

{'answer_relevancy': 0.0000, 'faithfulness': 0.0000, 'context_recall': 0.7619, 'context_precision': 0.8571}

In [75]:
df = result_prompt_2.to_pandas()
df.head()

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,What is Tech Innovators Inc.'s approach to wor...,I don't know.,[Q12: What is the process for handling workpla...,Tech Innovators Inc. has a zero-tolerance poli...,0.0,0.0,1.0,1.0
1,What resources should be added for new hires i...,I don't know.,[:check_mark:\natlassian-check_mark\n#FFFAE6\n...,Add resources for new hires in the onboarding ...,0.0,0.0,1.0,1.0
2,What training programs are offered in data sci...,I don't know.,"[ learn about SEO, social media marketing, and...","Courses covering financial analysis, budgeting...",0.0,0.0,0.0,0.0
3,What services does the Employee Assistance Pro...,I don't know.,[Q12: What is the process for handling workpla...,Employees can contact the Employee Assistance ...,0.0,0.0,1.0,1.0
4,What is the significance of identifying growth...,I don't know.,[ Self-assessment\nStart by thinking through y...,Identifying growth areas in self-assessment is...,0.0,0.0,1.0,1.0


## Template 3 evaluation

In [ ]:
# Generate all the answers for the questions in the dataset
# examples = client.list_examples(dataset_name="hr test")
answers_3 = []
for question in data_to_upload['question']:
    question_dict = {'question': question}
    answers = call_openai_template_3(question_dict)
    answers_3.append(answers['answer'])

In [ ]:
# update the dataset with answers
data_to_upload['answer_template_3'] = answers_3

In [ ]:
from datasets import Dataset

question = list(data_to_upload['question'])
answer = list(data_to_upload['answer_template_3'])
contexts = list(data_to_upload['contexts'])
ground_truth = list(data_to_upload['ground_truth'])

data_samples = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset_3 = Dataset.from_dict(data_samples)

In [ ]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result_prompt_3 = evaluate(
    dataset_3,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

result_prompt_3

Evaluating:   0%|          | 0/28 [00:00<?, ?it/s]

{'answer_relevancy': 0.0000, 'faithfulness': 0.0000, 'context_recall': 0.7619, 'context_precision': 0.8571}

In [ ]:
df = result_prompt_3.to_pandas()
df.head()

## Template 4 evaluation

In [95]:
# Generate all the answers for the questions in the dataset
# examples = client.list_examples(dataset_name="hr test")
answers_4 = []
for question in data_to_upload['question']:
    question_dict = {'question': question}
    answers = call_openai_template_4(question_dict)
    answers_4.append(answers['answer'])

In [96]:
# update the dataset with answers
data_to_upload['answer_template_4'] = answers_4

In [97]:
from datasets import Dataset

question = list(data_to_upload['question'])
answer = list(data_to_upload['answer_template_4'])
contexts = list(data_to_upload['contexts'])
ground_truth = list(data_to_upload['ground_truth'])

data_samples = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset_4 = Dataset.from_dict(data_samples)

In [98]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result_prompt_4 = evaluate(
    dataset_4,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

result_prompt_4

Evaluating:   0%|          | 0/28 [00:00<?, ?it/s]

{'answer_relevancy': 0.9756, 'faithfulness': 0.4053, 'context_recall': 0.7619, 'context_precision': 0.8571}

In [100]:
df = result_prompt_4.to_pandas()
df

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,What is Tech Innovators Inc.'s approach to wor...,Tech Innovators Inc. takes a proactive approac...,[Q12: What is the process for handling workpla...,Tech Innovators Inc. has a zero-tolerance poli...,0.999999,0.777778,1.000000,1.0
1,What resources should be added for new hires i...,Answer: For new hires in the onboarding proces...,[:check_mark:\natlassian-check_mark\n#FFFAE6\n...,Add resources for new hires in the onboarding ...,0.974871,0.000000,1.000000,1.0
2,What training programs are offered in data sci...,Tech Innovators Inc. offers a range of trainin...,"[ learn about SEO, social media marketing, and...","Courses covering financial analysis, budgeting...",0.989385,0.225806,0.000000,0.0
3,What services does the Employee Assistance Pro...,The Employee Assistance Program (EAP) provides...,[Q12: What is the process for handling workpla...,Employees can contact the Employee Assistance ...,0.994299,0.450000,1.000000,1.0
4,What is the significance of identifying growth...,Identifying growth areas in self-assessment is...,[ Self-assessment\nStart by thinking through y...,Identifying growth areas in self-assessment is...,0.988894,0.533333,1.000000,1.0
5,How do employee engagement and disengagement d...,Employee engagement and disengagement differ s...,"[Introduction\nAt Tech Innovators Inc., we bel...",Employee engagement and disengagement differ i...,0.920337,0.433333,1.000000,1.0
6,What is the purpose of adding a header image t...,Adding a header image to a Confluence space se...,[Create a stellar overview\nThe overview is th...,Adding a header image to a Confluence space en...,0.961312,0.416667,0.333333,1.0


## Template 5 evaluation

In [101]:
# Generate all the answers for the questions in the dataset
# examples = client.list_examples(dataset_name="hr test")
answers_5 = []
for question in data_to_upload['question']:
    question_dict = {'question': question}
    answers = call_openai_template_5(question_dict)
    answers_5.append(answers['answer'])

In [102]:
# update the dataset with answers
data_to_upload['answer_template_5'] = answers_5

In [103]:
from datasets import Dataset

question = list(data_to_upload['question'])
answer = list(data_to_upload['answer_template_5'])
contexts = list(data_to_upload['contexts'])
ground_truth = list(data_to_upload['ground_truth'])

data_samples = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    'ground_truth': ground_truth
}

dataset_5 = Dataset.from_dict(data_samples)

In [104]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result_prompt_5 = evaluate(
    dataset_5,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

result_prompt_5

Evaluating:   0%|          | 0/28 [00:00<?, ?it/s]

{'answer_relevancy': 0.0000, 'faithfulness': 0.0000, 'context_recall': 0.7619, 'context_precision': 0.8571}

In [105]:
df = result_prompt_5.to_pandas()
df

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,What is Tech Innovators Inc.'s approach to wor...,I don't know.,[Q12: What is the process for handling workpla...,Tech Innovators Inc. has a zero-tolerance poli...,0.0,0.0,1.000000,1.0
1,What resources should be added for new hires i...,I don't know.,[:check_mark:\natlassian-check_mark\n#FFFAE6\n...,Add resources for new hires in the onboarding ...,0.0,0.0,1.000000,1.0
2,What training programs are offered in data sci...,I don't know.,"[ learn about SEO, social media marketing, and...","Courses covering financial analysis, budgeting...",0.0,0.0,0.000000,0.0
3,What services does the Employee Assistance Pro...,I don't know.,[Q12: What is the process for handling workpla...,Employees can contact the Employee Assistance ...,0.0,0.0,1.000000,1.0
4,What is the significance of identifying growth...,I don't know.,[ Self-assessment\nStart by thinking through y...,Identifying growth areas in self-assessment is...,0.0,0.0,1.000000,1.0
5,How do employee engagement and disengagement d...,I don't know.,"[Introduction\nAt Tech Innovators Inc., we bel...",Employee engagement and disengagement differ i...,0.0,0.0,1.000000,1.0
6,What is the purpose of adding a header image t...,"I'm sorry, but I don't have information regard...",[Create a stellar overview\nThe overview is th...,Adding a header image to a Confluence space en...,0.0,0.0,0.333333,1.0
